In [1]:
from get_completion_1 import get_completion

In [2]:
q1 = "what was the count of the male candidates and female candidates who contested the 2019 maharashtra elections?" 
ans1 = """   SELECT candidate_sex, COUNT(*) AS candidate_count
FROM Maharashtra_elections_2019
GROUP BY candidate_sex;
"""

q2 = "what were the top three political prties which got the maximum number of votes, and how many votes did they get with respect to the parlimentary elections of 2024?"
ans2 = """
SELECT party_name_expanded, SUM(secured_votes) AS total_votes
FROM parlimentary_elections_2024
GROUP BY party_name_expanded
ORDER BY total_votes DESC
LIMIT 3;
"""

q3 = "find the political parties with the maximum number of female candidates and the count of female candidates in the 2019 Maharashtra elections"
ans3 = """
SELECT party_name, COUNT(*) AS female_candidate_count
FROM Maharashtra_elections_2019
WHERE candidate_sex = 'FEMALE'
GROUP BY party_name
ORDER BY female_candidate_count DESC;
"""

q4 = "count constituencies where BJP won in 2019 but lost in 2024 and where BJP won in 2024 but lost in 2019."
ans4 = """SELECT 
    CASE 
        WHEN p2019.secured_votes > p2024.secured_votes THEN 'BJP Won in 2019 but Lost in 2024'
        WHEN p2024.secured_votes > p2019.secured_votes THEN 'BJP Won in 2024 but Lost in 2019'
    END AS result_type,
    COUNT(*) AS constituencies_count
FROM 
    parlimentary_elections_2019 p2019
JOIN 
    parlimentary_elections_2024 p2024
ON 
    p2019.state_name = p2024.state_name 
    AND p2019.constituency_name = p2024.constituency_name
WHERE 
    p2019.party_name = 'BJP' 
    AND (p2019.secured_votes != p2024.secured_votes)  -- Ensuring there's a change in vote share
GROUP BY 
    result_type;
"""
q5 = "find the name of the political party that won the parliamentary elections in Maharashtra in both 2019 and 2024, along with the number of votes secured by the party in each year"
ans5 = """
SELECT 
    p2019.party_name AS party_name_2019,
    p2019.secured_votes AS votes_2019,
    p2024.party_name AS party_name_2024,
    p2024.secured_votes AS votes_2024
FROM 
    parlimentary_elections_2019 p2019
JOIN 
    parlimentary_elections_2024 p2024
ON 
    p2019.state_name = p2024.state_name 
    AND p2019.constituency_name = p2024.constituency_name
WHERE 
    p2019.party_name = p2024.party_name
ORDER BY 
    p2019.secured_votes DESC
LIMIT 1;

"""

In [3]:
schema_context = """
        Database Schema:
        1. Table: parlimentary_elections_2019
        Columns: state_name (TEXT), constituency_name (TEXT), assembly_constituency_name (TEXT), nota_votes (INTEGER), candidate_name (TEXT), party_name (TEXT), secured_votes (INTEGER)
        Data : 

        Andhra Pradesh,Aruku ,Palakonda (ST),3736,KISHORE CHANDRA DEO,TDP,54056
        Andhra Pradesh,Aruku ,Palakonda (ST),3736,Dr. KOSURI KASI VISWANADHA VEERA VENKATA SATYANARAYANA REDDY,BJP,1753
        Andhra Pradesh,Aruku ,Palakonda (ST),3736,GODDETI. MADHAVI,YSRCP,69588
        Andhra Pradesh,Aruku ,Palakonda (ST),3736,SHRUTI DEVI VYRICHERLA,INC,1327
        Andhra Pradesh,Aruku ,Palakonda (ST),3736,GANGULAIAH VAMPURU.,JnP,2987


        2. Table: Maharashtra_elections_2019
        Columns: state_name (TEXT), assembly_constituency_number (INTEGER), assembly_constituency_name (TEXT), candidate_name (TEXT), candidate_sex (TEXT), candidate_age (INTEGER), candidate_category (TEXT), party_name (TEXT), secured_votes (INTEGER), percentage_of_votes (REAL), total_voters (INTEGER), total_votes_in_state (INTEGER), PERCENT_VOTES_POLLED (REAL), candidate_position (INTEGER)
        Data : 

        Maharashtra,1,Akkalkuwa ,ADV. K. C. PADAVI,MALE,61,ST,INC,82770,41.25545786,278888,200628,71.93855598,1
        Maharashtra,1,Akkalkuwa ,AAMSHYA FULJI PADAVI,MALE,51,ST,SHS,80674,40.21073828,278888,200628,71.93855598,2
        Maharashtra,1,Akkalkuwa ,NAGESH DILVARSING PADVI,MALE,44,ST,IND,21664,10.79809398,278888,200628,71.93855598,3
        Maharashtra,1,Akkalkuwa ,NOTA,NOTA,0,NOTA,NOTA,4857,2.420898379,278888,200628,71.93855598,4
        Maharashtra,1,Akkalkuwa ,ADV. KAILAS PRATAPSING VASAVE,MALE,28,ST,AAAP,4055,2.021153578,278888,200628,71.93855598,5


        3. Table: parlimentary_elections_2024
        Columns: candidate_name (TEXT), party_name_expanded (TEXT), secured_votes (INTEGER), percentage_of_votes (REAL), state_name (TEXT), constituency_name (TEXT), party_name (TEXT)
        Data : 

        BISHNU PADA RAY,Bharatiya Janata Party,102436,50.58,Andaman & Nicobar Islands,Andaman & Nicobar Islands,BJP
        KULDEEP RAI SHARMA,Indian National Congress,78040,38.54,Andaman & Nicobar Islands,Andaman & Nicobar Islands,INC
        MANOJ PAUL,Andaman Nicobar Democratic Congress,8254,4.08,Andaman & Nicobar Islands,Andaman & Nicobar Islands,ANDC
        D AYYAPPAN,Communist Party of India (Marxist),6017,2.97,Andaman & Nicobar Islands,Andaman & Nicobar Islands,CPI (M)
        V.K. ABDUL AZIZ,Independent,2203,1.09,Andaman & Nicobar Islands,Andaman & Nicobar Islands,IND

    """

In [4]:
nl_query = "Find the count of votes secured by the political party BJP in the parlimentary elections of 2019 and parlimentary elections of 2024?"

In [5]:
prompt = f"""
    You are an expert in generating SQL queries from natural language queries.
    
    I want you to generate SQL query for the input question, which needs to be executed in SQLite3 in python. 

    Please keep in mind the schema of my database and the correct spellings of the table and column names.
    
    Just Give me one SQL query, nothing else. 
    
    I don't want any introduction, NO explanation, NO conclusion, just the correct SQL query.
    
    Please dont output backslashes. Use this database schema as context for generating SQL Queries which will answer the input question:

    {schema_context}

    #####

    Here are some EXAMPLES : 

    Input : {q1}
    Output : {ans1}

    Input : {q2}
    Output : {ans2}

    Input : {q3}
    Output : {ans3}

    Input : {q4}
    Output : {ans4}

    Input : {q5}
    Output : {ans5}

    #####

    Now generate the SQL query for the following question:

    Input : {nl_query}

"""

In [7]:
res = get_completion(prompt)
print(res)

SELECT COUNT(*) AS votes_count
FROM Maharashtra_elections_2019 p2019
JOIN
    Maharashtra_elections_2024 p2024
ON
    p2019.state_name = p2024.state_name
AND p2019.constituency_name = p2024.constituency_name
WHERE p2019.party_name = 'BJP'
ORDER BY p2019.secured_votes DESC;


In [1]:
from get_completion_1 import get_completion

res = get_completion("Where is Antarctica?")
print(res)

Antarctica is the southernmost continent on Earth, located almost entirely within the Antarctic Circle. It surrounds the South Pole and is situated in the Southern Hemisphere.

To be more specific, Antarctica is situated:

* At the bottom of the world, surrounding the South Pole
* In the Southern Ocean, which surrounds the continent
* South of the Atlantic, Indian, and Pacific Oceans
* East of South America, Africa, and Australia
* West of the Southern Ocean and the Drake Passage (which separates South America from Antarctica)

Antarctica is a frozen desert continent, covering an area of about 14 million square kilometers (5.4 million square miles), making it the fifth-largest continent on Earth. It is also the coldest, driest, and windiest continent, with an average temperature of around -50°C (-58°F) in winter and -12°C (10°F) in summer.

Despite its remote location, Antarctica is home to several research stations and a diverse range of wildlife, including penguins, seals, and whales